In [85]:
import pandas as pd
import numpy as np
import ela

In [86]:
zip_data = pd.read_csv('zipcode_data.csv')
gen_data = pd.read_csv('generation_data.csv')
stor_data = pd.read_csv('storage_data.csv')
stor_data.head()

,name,lat,lon,type2,type,power_kW,duration,status,usecase,city,state
0,PG&E Advanced Underground Compressed Air Energ...,37.917594,-121.171039,Compressed Air Storage,Electro-mechanical,300000,10,Announced,Electric Energy Time Shift,San Joaquin County,California
1,Next Gen CAES using Steel Piping - NYPA,40.728224,-73.794852,Compressed Air Storage,Electro-mechanical,9000,4.5,Announced,Black Start,Queens,New York
2,Lake Elsinore Advanced Pumped Storage,33.668077,-117.327262,Closed-loop Pumped Hydro Storage,Pumped Hydro Storage,500000,12,Announced,Frequency Regulation,Lake Elsinore,California
3,NYSEG Seneca/Watkins Glen CAES Project,42.415934,-76.896832,Compressed Air Storage,Electro-mechanical,0,n/a,Announced,Frequency Regulation,Reading,New York
4,Con Edison/Eos Energy Storage Distributed Ener...,40.714353,-74.005973,Metal Air Battery,Electro-chemical,1000,6,Announced,Stationary Transmission/Distribution Upgrade D...,New York,New York


In [157]:
def test_get_latlon_from_zip():
    """Tests for checking the latitude-longitude for the entered zip code"""
    assert ela.get_latlon_from_zip(98105)== (47.66377, -122.30118), "Zipcode entered incorrectly!"
    assert type(ela.get_latlon_from_zip(81059))==tuple, "Lat-Lon is not in the expected format"
    assert len(ela.get_latlon_from_zip(81059))==2 , "The length of the tuple is incorrect"
    return

In [158]:
def test_get_state_from_zip():
    """Test for checking the state for the entered zipcode"""
    assert type(ela.get_state_from_zip(11206))==str,"Output is not a string"
    assert ela.get_state_from_zip(98105)=='WA', "Zipcode does not match the state"
    assert len(ela.get_state_from_zip(98105))==2 ,"The length of the string is incorrect"
    return


In [159]:
def test_get_closest_facility():
    test_df = pd.DataFrame('ela.gen_data')
    assert len(len(ela.get_closest_facility( ela.get_latlon_from_zip(98105),'gen').state))==2,"The state name is a stirng of 2 characters"
    assert type(ela.get_closest_facility( ela.get_latlon_from_zip(98105),'gen').lat)==float,"The latitude data is not precise"
    assert ela.get_closest_facility( ela.get_latlon_from_zip(98105),'gen').capacity_MW> ela.get_closest_facility( ela.get_latlon_from_zip(98034),'gen').production_GWh, "Production is lesser than capacity"
    return


In [160]:
ela.get_predicted_type( ela.get_latlon_from_zip(98105), 'stor')

'Electro-chemical'

In [161]:
def test_get_predicted_type():
    assert type(ela.get_predicted_type(ela.get_latlon_from_zip(98105),'stor'))==str ,"Output is not a string"
    assert len(ela.get_predicted_type( ela.get_latlon_from_zip(98033), 'gen'))==3,"The energy resource of the plant is not oil or gas"
    assert len(ela.get_predicted_type( ela.get_latlon_from_zip(98105), 'stor'))==16, "The storage facility is not electro-chemical"
    return

In [162]:
ela.get_state_breakdown(ela.get_state_from_zip(98033),gen_data)

,HYDRO,WIND,BIOMASS,GAS,OIL,COAL,OTHF,NUCLEAR
WA,53%,15%,13%,11%,3%,1%,0%,0%


In [163]:
ela.get_state_breakdown(ela.get_state_from_zip(98033),gen_data).HYDRO>ela.get_state_breakdown(ela.get_state_from_zip(98033),gen_data).WIND

WA    True
dtype: bool

In [164]:
def test_get_state_breakdown():
    A=ela.get_state_breakdown(ela.get_state_from_zip(98033),gen_data).HYDRO
    B=ela.get_state_breakdown(ela.get_state_from_zip(98033),gen_data).WIND
    C=ela.get_state_breakdown(ela.get_state_from_zip(98033),gen_data).GAS
    D=ela.get_state_breakdown(ela.get_state_from_zip(98033),gen_data).NUCLEAR
    assert A>B, "Energy generated from wind is more than hydro"
    assert D==0,"Energy is generated from nuclear resources"
    assert C<B, "Energy generated from gas is more then wind"
    return

In [165]:
ela.get_energy_breakdown(gen_data, location='US')

,GAS,SOLAR,HYDRO,WIND,OIL,BIOMASS,COAL,OTHF,GEOTHERMAL,NUCLEAR,OTHER,OFSL
US,22%,17%,17%,12%,10%,9%,6%,1%,0%,0%,0%,0%


In [166]:
ela.get_energy_breakdown(gen_data, location='US').SOLAR==ela.get_energy_breakdown(gen_data, location='US').HYDRO,"energy generated from hydro and solar are different"

(US    True
 dtype: bool, 'energy generated from hydro and solar are different')

In [167]:
def test_get_energy_breakdown(facility_data, location='US'):
    assert ela.get_energy_breakdown(gen_data, location='US').SOLAR==ela.get_energy_breakdown(gen_data, location='US').HYDRO,"energy generated from hydro and solar are different"
    assert len(list(ela.get_energy_breakdown(gen_data, location='US')))==12,"total number of resources considered are incorrect"
    assert ela.get_energy_breakdown(gen_data, location='US').GAS>ela.get_energy_breakdown(gen_data, location='US').COAL,"energy generated from coal is more than gas"
    return

In [168]:
ela.state_type(gen_data)

,BIOMASS,COAL,GAS,GEOTHERMAL,HYDRO,NUCLEAR,OFSL,OIL,OTHER,OTHF,SOLAR,WIND
AK,0%,3%,9%,0%,22%,0%,0%,58%,0%,1%,0%,2%
AL,13%,15%,32%,0%,30%,2%,0%,3%,0%,2%,0%,0%
AR,15%,7%,38%,0%,30%,1%,0%,4%,0%,0%,1%,0%
AZ,4%,4%,25%,0%,9%,0%,0%,1%,1%,0%,47%,4%
CA,9%,0%,23%,3%,19%,0%,0%,1%,0%,0%,32%,9%
CO,2%,8%,18%,0%,28%,0%,0%,6%,0%,1%,17%,16%
CT,10%,1%,36%,0%,16%,1%,1%,26%,0%,0%,6%,0%
DC,50%,0%,50%,0%,0%,0%,0%,0%,0%,0%,0%,0%
DE,11%,3%,26%,0%,0%,0%,0%,19%,3%,3%,26%,3%
FL,20%,6%,40%,0%,1%,1%,0%,10%,3%,4%,12%,0%


In [169]:
def test_state_type(facility_data):
    test_df=pd.DataFrame('gen_data')
    assert ela.state_type(gen_data).OIL[0]>ela.state_type(gen_data).OIL[1],"energy generated using oil as resource is greater in alabama than arkansas"
    assert ela.state_type(gen_data).HYDRO.WA>ela.state_type(gen_data).SOLAR.WA,"energy geneeration from hydro plants is less than solar plants in WA "
    assert len(ela.state_type(gen_data))==51," incorrect number of states"
    return

In [113]:
ela.state_type(gen_data).HYDRO.WA>ela.state_type(gen_data).SOLAR.WA

True

In [ ]:
def test_graph_state_breakdown():
    test_df=pd.DataFrame('gen_data')
    assert 